#Import libraries

In [ ]:
import requests
from bs4 import BeautifulSoup
from google.colab import files
import pandas as pd

In [ ]:
pip install selenium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.2/467.2 kB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.1 MB/s eta 0:00:00


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import TimeoutException, NoSuchElementException

#Dou


In [ ]:
def show_all_vacancies(page_url):

    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')

    chrome_options.binary_location = '/Applications/Google\ Chrome.app/Contents/MacOS/Google\ Chrome'

    driver = webdriver.Chrome(options=chrome_options)

    driver.get(page_url)
    all_content = ""

    while True:
        try:
            #button = driver.find_element(By.CSS_SELECTOR, "#vacancyListId > div > a")

            button = WebDriverWait(driver, 1).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, "#vacancyListId > div > a"))
)
            button.click()

            if 'display: none;' in button.get_attribute('style'):
              break
            else:
              pass


        except TimeoutException:
            break  # All vacancies loaded
        except StaleElementReferenceException:
            continue  # Catch and retry
        except Exception as e:
            #print(f"An error occurred: {str(e)}")
            break  # Break on other errors
        #except NoSuchElementException:
          #print("Button not found")

    all_content = driver.page_source
    soup = BeautifulSoup(all_content, 'html.parser')

    driver.quit()

    return soup

In [ ]:
def dou_jobs(soup):
    job_offers = []
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')

    chrome_options.binary_location = '/Applications/Google\ Chrome.app/Contents/MacOS/Google\ Chrome'

    driver = webdriver.Chrome(options=chrome_options)

    if soup:
        for job_card in soup.select('.l-vacancy'):
            title = job_card.select_one('.vt').get_text(strip=True) if job_card.select_one('.vt') else None
            company = job_card.select_one('.company').get_text(strip=True) if job_card.select_one('.company') else None
            location = job_card.select_one('.cities.bi.bi-geo-alt-fill').get_text(strip=True) if job_card.select_one('.cities.bi.bi-geo-alt-fill') else None
            salary = job_card.select_one('.salary').get_text(strip=True) if job_card.select_one('.salary') else None
            date = job_card.select_one('.date').get_text(strip=True) if job_card.select_one('.date') else None
            url = job_card.select_one('.vt')['href'] if job_card.select_one('.vt') else None

            driver.get(url)  # Navigate to the job's URL
            descr = driver.find_element("class name", "vacancy-section")
            description = descr.text

            job_details = {
                'Title': title,
                'Company': company,
                'Location': location,
                'Salary': salary,
                'Date': date,
                'Url': url,
                'Description': description
            }

            job_offers.append(job_details)

    return pd.DataFrame(job_offers)

In [ ]:
df = dou_jobs(show_all_vacancies("https://gamedev.dou.ua/jobs/"))

In [ ]:
df.head()

,Title,Company,Location,Salary,Date,Url,Description
0,Senior Game Designer (Slot Games),Sigma Software,None,None,22 квітня 2024,https://gamedev.dou.ua/jobs/companies/sigma-so...,"Ти творчий та креативний геймдизайнер, який пр..."
1,NOC Team Leader,Playtika,None,None,3 травня 2024,https://gamedev.dou.ua/jobs/companies/playtika...,We are looking for an enthusiastic person to b...
2,Full Stack Developer,Playtika,None,None,3 травня 2024,https://gamedev.dou.ua/jobs/companies/playtika...,We are looking or a Full Stack Developer to jo...
3,Senior 2D artist,Qubit Labs,None,None,3 травня 2024,https://gamedev.dou.ua/jobs/companies/qubit-la...,Mobile Game Development Company is looking for...
4,Middle ASO Specialist,Bini Games,None,None,3 травня 2024,https://gamedev.dou.ua/jobs/companies/bini-gam...,Bini Games — це українська продуктова компанія...


In [ ]:
df.to_csv('dou_04-05.csv', index=False)
files.download('dou_04-05.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#Djinni

In [ ]:
def dj_full_text(url):

  response = requests.get(url)
  soup = BeautifulSoup(response.text, 'html.parser')

  job_description = soup.select_one('.mb-4').text

  return job_description

In [ ]:
def djinni_jobs(url):

  '''
  Description: gets main data on job vacancies from one Djinni page

  Input: url of the page to parse

  Ouput: pandas DataFrame containing parsed data in string format

       ^Job title
       ^company name
       ^location
       ^salary
       ^full job description link
       ^occupation
       ^level of experince
       ^English level
       ^company type
       ^date of publishment
       ^views on vacancy
       ^number of candidaties applyed
  '''

  job_offers = []

  # Define lists of words for occupation, experience-year, English level
  occup_w = ['Office або Remote', 'Тільки віддалено', 'Гібридна робота', 'Тільки офіс',
             'Full Remote','Office or Remote','Office','Hybrid','Office Work','Hybrid Remote']
  exp_w = ['досвіду', 'роки','of experience','No experience']
  eng_w = ['Beginner/Elementary', 'Pre-Intermediate', 'Intermediate', 'Upper-Intermediate', 'Advanced/Fluent']
  comp_t_w = ['Product','Oursource','Outstaff','Agency']

  response = requests.get(url)
  soup = BeautifulSoup(response.text, 'html.parser')

  if soup:
    for job_card in soup.select('.list-jobs__item'):

      title = job_card.select_one('.job-list-item__link').text.strip() if job_card.select_one('.job-list-item__link') else None
      url = 'https://djinni.co' + job_card.select_one('.job-list-item__link')['href'] if job_card.select_one('.job-list-item__link') else None
      descr = dj_full_text(url)
      salary = job_card.select_one('.public-salary-item').text.strip() if job_card.select_one('.public-salary-item') else None

      date = job_card.select_one('span.text-muted').find('span', class_='mr-2 nobr')['title'].split(' ')[1] if job_card.select_one('span.text-muted').find('span', class_='mr-2 nobr') else None
      view = job_card.select_one('span.text-muted').find_all('span', class_='mr-2')[1]['title'].split(' ')[0] if job_card.select_one('span.text-muted').find_all('span', class_='mr-2')[1] else None
      apply = job_card.select_one('span.text-muted').find_all('span', class_='mr-2')[2]['title'].split(' ')[0] if job_card.select_one('span.text-muted').find_all('span', class_='mr-2')[2] else None

      company = job_card.select_one('.d-flex .job-list-item__pic + a').text.strip() if job_card.select_one('.d-flex .job-list-item__pic + a') else None
      location = job_card.select_one('.location-text').text.strip().replace('\n',' ') if job_card.select_one('.location-text') else None

      eng = None
      exp = None
      occup = None
      comp_type = None

      for txt in job_card.select_one('div.job-list-item__job-info').find_all('span', class_='nobr'):
              t = txt.text.replace(' · ','')
              if t in occup_w:
                occup = t
              elif t in eng_w:
                eng = t
              elif any(word in t for word in exp_w):
                exp = t
              elif t in comp_t_w:
                comp_type = t
              else:
                print(t)

      job_details = {
                'Title': title,
                'Company': company,
                'Location': location,
                'Salary': salary,
                'Url': url,
                'Occupation': occup,
                'Experience': exp,
                'Eng level': eng,
                'Company type': comp_type,
                'Date': date,
                'Views': view,
                'Applied': apply,
                'Description': descr
            }

      job_offers.append(job_details)

  return pd.DataFrame(job_offers)


In [ ]:
def djinni_all_pages(url, max_pages=100):
    job_offers_all_pages = []

    for page_num in range(1, max_pages + 1):
        current_url = f'{url}&page={page_num}'
        response = requests.get(current_url)
        if response.url == 'https://djinni.co/jobs/':
          break
        job_offers_page = djinni_jobs(current_url)

        job_offers_all_pages.append(job_offers_page)

    return pd.concat(job_offers_all_pages, ignore_index=True)

In [ ]:
dt = djinni_all_pages('https://djinni.co/jobs/?all-keywords=Game&keywords=Game')

In [ ]:
dt.head()

,Title,Company,Location,Salary,Url,Occupation,Experience,Eng level,Company type,Date,Views,Applied,Description
0,Trainee Game Designer,Болі Віяс,Ukraine (Kyiv),$600-850,https://djinni.co/jobs/648348-trainee-game-des...,Office Work,No experience,None,Product,04.05.2024,49,9,\n Ми шукаємо молодого та амбіційного к...
1,Platform Product manager,PRO Gaming Software,Czechia,None,https://djinni.co/jobs/648343-platform-product...,Full Remote,3 years of experience,Upper-Intermediate,Product,04.05.2024,5,1,\n We are an international iGaming comp...
2,Product Manager,PRO Gaming Software,Spain,None,https://djinni.co/jobs/648303-product-manager/,Full Remote,3 years of experience,Intermediate,Product,03.05.2024,19,4,\n We are an iGaming company that was f...
3,HR Generalist (with recruitment),Tom Horn Gaming,"Ukraine, Poland, Portugal, Spain, ...",None,https://djinni.co/jobs/648225-hr-generalist-wi...,Full Remote,3 years of experience,Advanced/Fluent,Product,03.05.2024,50,12,\n We are seeking an experienced HRGene...
4,Lead User Acquisition Manager,Vira Games,Ukraine,None,https://djinni.co/jobs/648200-lead-user-acquis...,Full Remote,5 years of experience,None,Product,03.05.2024,13,1,"\nVIRA GAMES — незалежна, українська ігрова ко..."


In [ ]:
dt.to_csv('djinni_04-05.csv', index=False)
files.download('djinni_04-05.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>